In [3]:
import numpy as np
import matplotlib.pyplot as plt
import math
import gym
import random
import torch

from gym import Env
from gym.spaces import Discrete, Box


In [4]:
env = gym.make('CartPole-v0')
states = env.observation_space.shape[0]
actions = env.action_space.n

c:\Python311\Lib\site-packages\gym\envs\registration.py:555: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(


In [5]:
episodes = 100
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = random.choice([0,1])
        n_state, reward, done, truncated,info = env.step(action)
        score += reward
    print("Episode {} Score: {}".format(episode, score))

Episode 1 Score: 19.0
Episode 2 Score: 15.0
Episode 3 Score: 49.0
Episode 4 Score: 28.0
Episode 5 Score: 14.0
Episode 6 Score: 18.0
Episode 7 Score: 19.0
Episode 8 Score: 12.0
Episode 9 Score: 12.0
Episode 10 Score: 16.0
Episode 11 Score: 16.0
Episode 12 Score: 11.0
Episode 13 Score: 10.0
Episode 14 Score: 17.0
Episode 15 Score: 11.0
Episode 16 Score: 18.0
Episode 17 Score: 45.0
Episode 18 Score: 18.0
Episode 19 Score: 9.0
Episode 20 Score: 26.0
Episode 21 Score: 16.0
Episode 22 Score: 12.0
Episode 23 Score: 15.0
Episode 24 Score: 11.0
Episode 25 Score: 10.0
Episode 26 Score: 23.0
Episode 27 Score: 13.0
Episode 28 Score: 17.0
Episode 29 Score: 14.0
Episode 30 Score: 25.0
Episode 31 Score: 9.0
Episode 32 Score: 15.0
Episode 33 Score: 31.0
Episode 34 Score: 52.0
Episode 35 Score: 12.0
Episode 36 Score: 19.0
Episode 37 Score: 17.0
Episode 38 Score: 53.0
Episode 39 Score: 21.0
Episode 40 Score: 23.0
Episode 41 Score: 16.0
Episode 42 Score: 24.0
Episode 43 Score: 14.0
Episode 44 Score: 31.0

c:\Python311\Lib\site-packages\gym\envs\classic_control\cartpole.py:211: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym("CartPole-v0", render_mode="rgb_array")
  gym.logger.warn(
c:\Python311\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


## Custom environment

In [6]:
# Define the WaterTank environment
class WaterTank(Env):
    def __init__(self, setpoint = 10.0, dt = 1e-3):
        """
        The tank is governed by the following first-order differential equation
        
        A(dH/dt) = bV - a*sqrt(H)

        The states of the system are: H
        The inputs of the system are: v
        The outputs of the system are: H

        H:  Height of water
        V: voltage applied to the pump

        """

        # Possible actions
        self.action_space = Box(
            low = np.array([0.0]),
            high = np.array([120.0])
        ) # voltage values
        
        # Water level temperature array
        self.observation_space = Box(low = np.array([1e-6]), high = np.array([30.0]))

        # Tank parameters
        self.H0 = 100 + np.random.uniform(-100, 100) # initial level
        self.A = 20 # cross sectional area of the tank

        # Initialize current water level
        self.H = self.H0 # meters

        # Set desired level
        self.setpoint = setpoint

        # Input valve parameters
        self.b = 5 # Constant related to the flow rate into the tank
        self.V = 5 + np.random.uniform(-5, 10) # Initial voltage of the pump
        self.P = self.V*self.b

        # Output valve parameters
        self.a = 2 # Constant related to the flow rate out of the tank
        self.Q = self.a*math.sqrt(self.H) # Output flow depends of pressure which depends of water height

        self.dt = dt # each iteration is a 10ms time-step

        # Initial water level, inflow and outflow
        self.state = self.H


    def step(self, action):
        
        # action is the voltage

        # Update inflow
        self.P = self.b*self.V

        # Output flow
        self.Q = self.a*math.sqrt(self.H)

        # Update tank level using trapezoidal method
        Hold = self.H
        Hnew = Hold + (self.dt/2.0)*((self.P - self.Q)/self.A)
        #self.H += ((self.P - self.Q)/self.A) * self.dt # Trapezoidal method
        self.H = Hnew
        if self.H <= 0:
            self.H = 1e-6

        # Compute reward as the negative abs difference between current level and setpoint
        
        reward = -abs(self.H - self.setpoint)  # Negative absolute difference as reward

        done = abs(self.H - self.setpoint) < 3

        info = {}
        self.state = np.array([self.H, self.V, self.Q])
        return self.state, reward, done, False, info
    
    def render(self):
        pass
    
    def reset(self):
        # Initialize current water level
        self.H0 = 100 + np.random.uniform(-100, 100) # initial level
        self.H = self.H0 # meters

        # Input valve parameters
        self.V = 5 + np.random.uniform(-5, 10) # Initial voltage of the pump
        self.P = self.V*self.b

        # Output valve parameters
        self.Q = self.a*math.sqrt(self.H) # Output flow depends of pressure which depends of water height

        reward = -abs(self.H - self.setpoint)  # Negative absolute difference as reward

        self.state = np.array([self.H, self.V, self.Q])

        return self.state



In [7]:
env = WaterTank()

ValueError: Box shape is inferred from low and high, expect their types to be np.ndarray, an integer or a float, actual type low: <class 'torch.Tensor'>, high: <class 'torch.Tensor'>

In [11]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, truncated,info = env.step(action)
        score += reward
    print("Episode {} Score: {}".format(episode, score))

Episode 1 Score: -83723.1303793332
Episode 2 Score: -46359180337.74294
Episode 3 Score: -358306725.11723787
Episode 4 Score: -7064363.9681306025


## NN Model

In [20]:
# Define the Deep Q-Network (DQN) model
class DQN(nn.Module):
    def __init__(self, input_dim, output_dim, layers = [32, 64, 128, 256]):
        super(DQN, self).__init__()

        # Create input layer
        self.input = nn.Linear(input_dim, layers[0])
        self.output = nn.Linear(layers[-1], output_dim)

        # Hidden layers
        self.layers = []
        for i in range(len(layers)-1):
            self.layers.append(
                nn.Linear(layers[i], layers[i+1])
            )

        #self.hidden = nn.Sequential(*self.layers)

    def forward(self, x):

        x = torch.relu(self.input(x))

        #x = torch.relu(self.hidden(x))
        for layer in self.layers:
            x = torch.relu(layer(x))

        return self.output(x) # linear output

In [23]:
states = env.observation_space.shape[0]
actions = env.action_space.n

model = DQN(states, actions)
model(torch.tensor([5], dtype=torch.float32))

tensor([ 0.0110,  0.0112,  0.0381,  0.0379, -0.1070], grad_fn=<ViewBackward0>)

In [32]:
# Initialize the environment and DQN model
env = WaterTank(
    setpoint = 3
)
state_dim = env.state_variables_dim
action_dim = env.actions_dim
model = DQN(state_dim, action_dim, [128, 64, 32]).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

# Define simulation time
tf = 5 # in seconds
N = int(math.ceil(tf/env.dt))

# Define training parameters
num_episodes = 50 
gamma = 0.99
max_iters_per_episode = N # equivalent to tf seconds
n_time_steps = N

rewards = []
water_levels = []
input_flows = []

print(f"Episodes {num_episodes}, steps per episode {max_iters_per_episode}")


Episodes 50, steps per episode 5000


In [33]:
# Test model and env
state, reward = env.reset()
state = torch.tensor(state, dtype = torch.float32, device = device)
q_values = model(state)
q_values

tensor([0.1315, 0.0390], grad_fn=<ViewBackward0>)

In [34]:
# Train the model
for episode in range(num_episodes):
    #state = torch.tensor(env.step(0)[0], dtype=torch.float32)
    state, reward = env.reset()
    state = torch.tensor(state, dtype=torch.float32, device = device)
    total_reward = 0

    #mean_episode_level = 0
    #mean_episode_inflow = 0
    #mean_episode_outflow = 0

    for i in range(n_time_steps):
    #while not done:
        ti = (i-1)*env.dt # current time

        q_values= model(state) # get model values
        
        #valve = torch.relu(q_values[-1]).item() # Valve voltage

        #q_values = q_values[:2] # Actions
        #q_values = torch.softmax(q_values, dim = 0) # Compute actions probabilities
        action = torch.argmax(q_values).item() # Pick action with highest prob
        
        next_state, reward = env.step(action) # Compute next state
        
        next_state_tensor = torch.tensor(next_state, dtype=torch.float32, device = device)
        
        target_q_value = reward + gamma * torch.max(model(next_state_tensor))
        
        loss =  criterion(q_values[action], target_q_value)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        state = next_state_tensor
        total_reward += reward

        #mean_episode_level += state[0]
        #mean_episode_inflow += state[1]
        #mean_episode_outflow += state[2]

    #mean_episode_level /= max_iters_per_episode
    #mean_episode_inflow /= max_iters_per_episode
    #mean_episode_outflow /= max_iters_per_episode

    print(f'Episode {episode + 1} - Total Reward: {total_reward}')
    rewards.append(total_reward)
    #water_levels.append(mean_episode_level)
    #input_flows.append(mean_episode_inflow)

#fig, axes = plt.subplots(nrows = 1, ncols = 3)
plt.figure(1)
plt.plot(np.arange(len(rewards))*env.dt, rewards)
#axes[1].plot(np.arange(len(water_levels))*env.dt, water_levels)
#axes[2].plot(np.arange(len(input_flows))*env.dt, input_flows)
plt.show()




Episode 1 - Total Reward: -153105.36849028646
Episode 2 - Total Reward: -2164770.4303629817
Episode 3 - Total Reward: -2611235.5538996058
Episode 4 - Total Reward: -2611235.5538996058
Episode 5 - Total Reward: -2293736.718196942
Episode 6 - Total Reward: -14900.338300455654
Episode 7 - Total Reward: -14900.338300455654
Episode 8 - Total Reward: -14900.338300455654
Episode 9 - Total Reward: -14900.338300455654
Episode 10 - Total Reward: -14900.338300455654
Episode 11 - Total Reward: -14900.338300455654
Episode 12 - Total Reward: -14900.338300455654
Episode 13 - Total Reward: -14900.338300455654
Episode 14 - Total Reward: -14900.338300455654
Episode 15 - Total Reward: -14900.338300455654
Episode 16 - Total Reward: -14900.338300455654


KeyboardInterrupt: 

In [12]:
state

tensor([4.0872, 4.6896, 4.0410])

In [121]:
# Deploy the model
state_deploy = torch.tensor(env.step(0)[0], dtype=torch.float32)
done_deploy = False

# Create a figure to plot the water level
fig, ax = plt.subplots()


while not done_deploy:
    q_values_deploy = model(state_deploy)
    action_deploy = torch.argmax(q_values_deploy).item()

    next_state_deploy, reward_deploy = env.step(action_deploy)

    state_deploy = torch.tensor(next_state_deploy, dtype=torch.float32)

    env.current_level += next_state_deploy[1]  # Update current level based on action taken

    env_render_info = f"Current Level: {env.current_level}, Reward: {reward_deploy}"
    
    print(env_render_info)

TypeError: WaterTank.step() missing 1 required positional argument: 'valve_perc'